master_course.csvを読み込んで、検証

In [29]:
PLAYER = "2878"

In [33]:
import pandas as pd

# 読み込み
path = r"C:\Users\user\Desktop\Git\ark-blueprints\data\processed\course\master_course.csv"
df = pd.read_csv(path, dtype=str, encoding="utf-8-sig")

# 日付をdatetime化（失敗はNaT）
df["date"] = pd.to_datetime(df.get("date"), errors="coerce")

# フィルタ：PLAYER かつ entry=1
mask = (df["player_id"].astype(str) == PLAYER) & (pd.to_numeric(df["entry"], errors="coerce") == 5)
sub = df.loc[mask, ["date", "race_id", "rank"]].copy()

# 日付で昇順に並べ、直近10走を取得
sub = sub.sort_values("date").tail(11)

# 出力
for _, r in sub.iterrows():
    d = r["date"].strftime("%Y-%m-%d") if pd.notna(r["date"]) else "NaT"
    print(d, r["race_id"], r["rank"])


2025-05-26 202505262004 3
2025-06-05 202506052401 3
2025-06-17 202506171610 5
2025-07-23 202507232205 2
2025-08-09 202508092411 5
2025-08-28 202508281102 3
2025-09-13 202509131808 3


In [ ]:
# -*- coding: utf-8 -*-
# master.csv（rank, entry, ST を使用）→ 直近n走の率/平均を付与 → master_course.csv

import pandas as pd
import numpy as np

IN_PATH  = r"C:\Users\user\Desktop\Git\ark-blueprints\data\processed\master.csv"
OUT_PATH = r"C:\Users\user\Desktop\Git\ark-blueprints\data\processed\master_course.csv"

# ===== 固定前提の列名 =====
PLAYER_COL = "player_id"
ENTRY_COL  = "entry"           # コース（進入後）
RANK_COL   = "rank"            # 着順（1,2,3,...）
ST_COL     = "ST"              # スタートタイミング
DATE_COL   = "date"
RACE_COL   = "race_id"

# ===== パラメータ =====
N_LAST = 10  # 直近n走

# ---- 読み込み ----
df = pd.read_csv(IN_PATH, parse_dates=[DATE_COL])

# 必須列チェック（厳密）
required = {PLAYER_COL, ENTRY_COL, RANK_COL, ST_COL, DATE_COL, RACE_COL}
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"必須列が不足しています: {missing}")
# 時系列ソート（リーク防止の土台）
df = df.sort_values([DATE_COL, RACE_COL]).reset_index(drop=True)

# ちょうど1位/2位/3位のフラグ（以内ではない）
df["exact1_flag"] = (df[RANK_COL] == 1).astype(int)
df["exact2_flag"] = (df[RANK_COL] == 2).astype(int)
df["exact3_flag"] = (df[RANK_COL] == 3).astype(int)
# グループ（選手×entry）
g = df.groupby([PLAYER_COL, ENTRY_COL], sort=False)

# 当該レースを含めないため shift(1)
prev_cols = ["exact1_flag", "exact2_flag", "exact3_flag", ST_COL]
for col in prev_cols:
    df[f"_{col}_prev"] = g[col].shift(1)

def add_lastn_rate(col_key: str, out_key: str, n: int):
    # 直近n走の成功数と分母（有効件数）→ 比率
    wins_n = g[f"_{col_key}_prev"].rolling(n, min_periods=1).sum()
    cnts_n = g[f"_{col_key}_prev"].rolling(n, min_periods=1).count()
    df[f"{out_key}_rate_last{n}_pe"] = (wins_n / cnts_n).reset_index(level=[0,1], drop=True)
    df[f"{out_key}_cnt_last{n}_pe"]  = cnts_n.reset_index(level=[0,1], drop=True)

# 1位/2位/3位の “ちょうど” 率
add_lastn_rate("exact1_flag", "exact1", N_LAST)  # 1着率（exact）
add_lastn_rate("exact2_flag", "exact2", N_LAST)  # 2着率（exact）
add_lastn_rate("exact3_flag", "exact3", N_LAST)  # 3着率（exact）

# ST の直近n走 統計
st_mean = g[f"_{ST_COL}_prev"].rolling(N_LAST, min_periods=1).mean()
st_std  = g[f"_{ST_COL}_prev"].rolling(N_LAST, min_periods=2).std()
df[f"ST_mean_last{N_LAST}_pe"] = st_mean.reset_index(level=[0,1], drop=True)
df[f"ST_std_last{N_LAST}_pe"]  = st_std.reset_index(level=[0,1], drop=True)

# ...（前半はそのまま）...

# 一時列の削除（_prev 系のみ）
df.drop(columns=[f"_{c}_prev" for c in prev_cols], inplace=True)

# 出力
df.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print(
    f"[OK] wrote: {OUT_PATH}  rows={len(df):,}  lastN={N_LAST}  "
    f"group=({PLAYER_COL},{ENTRY_COL})  cols_added=[exact1/exact2/exact3]_rate/_cnt, ST_mean/std"
)
